In [ ]:
from importlib import reload
import ipynb.fs.full.viewer
reload(ipynb.fs.full.viewer)
from ipynb.fs.full.viewer import Viewer, root, bucket
import pickle
from pprint import pprint
from dotdict import dotdict
from functools import reduce

from typing import *
import matplotlib.pyplot as plt
import random
import PIL
from pathlib import Path
import json
from collections import defaultdict
from vivid.utils import timer
import subprocess
from subprocess import PIPE
from tqdm import tqdm
from time import time
from functools import lru_cache

In [ ]:
old = set(Path(root/"danbooru2019/512px.dat").read_text().strip().split())
safe =  set((root/"danbooru2020/512px.dat").read_text().strip().split())

fire_eggs = dict()
for k in "duplicates is_deleted no_humans not-image photo text_only_page".split():
    fire_eggs[k] = set((root/f"danbooru2020/Danbooru2019/reduce/{k}.txt").read_text().strip().split())
    
ge512 =  set((root/"danbooru2020/ge512.ids").read_text().strip().split())
ge1024 =  set((root/"danbooru2020/ge1024.ids").read_text().strip().split())

i = 0
if not "ids" in globals():
    with timer(prefix="read_id_pkl"): # 311[s]
        with open((root/f"{i}_id.pkl"), "rb") as f:
            ids = pickle.load(f)
if not "tags" in globals():
    with timer(prefix="read_tag_pkl"): # 58[s]
        with open((root/f"{i}_tag.pkl"), "rb") as f:
            tags = pickle.load(f)

In [ ]:
@lru_cache(maxsize=None)
def t(s):
    if "|" in s:
        retval = reduce(set.__or__, map(tags.get, s.split("|")))
    else:
        retval = reduce(set.__and__, map(tags.get, s.split("&")))
    return retval
def id2fname(_id, prefix="512white"):
    return str(root/f"danbooru2020/{prefix}/{bucket(_id)}/{_id}.png")

In [ ]:
#solo = t("1girl|solo|1boy")
solo = t("1girl|solo|1boy")
standing = t("standing&full_body")
white = t("white_background|transparent_background")
exc = t('|'.join([
    "2boys|2girls|3boys|3girls|4boys|4girls|5boys|5girls|6+boys|6+girls|multiple_boys|multiple_girls",
    "car|bicycle",
    "no_humans|arachne|pokemon_(creature)|octoling|monster_girl|isabelle_(animal_crossing)|animal_nose|animal_hug",
    "robot|cyborg|mecha_musume|machinery|doll_joints|crewmate_(among_us)",
    "blue_skin|purple_skin|wrinkled_skin|green_skin|two-tone_skin",
    "monochrome|lineart|text_only_page|photo-referenced|3d|unconventional_media|comic",
    "multiple_views|variations|zoom_layer",
    "full_armor|weapon|chibi|cannon|mask",
    "back|ass|from_behind|from_above|bent_over|dutch_angle|standing_split",
    "translation_request",
    "fox_tail",
    "otter_costume",
]))|reduce(set.__or__, fire_eggs.values())|t("1girl&1boy")
#v3 = (safe&solo&standing&white&ge1024)-exc
v3 = set()|solo # id(v3) != id(solo)
print("{:,}\t {}".format(len(v3), "solo"))
v3 &= standing
print("{:,}\t {}".format(len(v3), "standing"))
v3 &= safe
print("{:,}\t {}".format(len(v3), "safe"))
v3 &= ge1024
print("{:,}\t {}".format(len(v3), "ge1024"))
v3 &= white
print("{:,}\t {}".format(len(v3), "white"))
v3 -= exc
print("{:,}\t {}".format(len(v3), "exc"))

In [ ]:
viewer_v3 = Viewer(map(id2fname, v3), N=12*6-1, name="v3")
viewer_v3

In [ ]:
for k in ids["4203535"]["tags"]:
    print(k["name"])

In [ ]:
Viewer(map(id2fname, tags[""]))

In [ ]:
Viewer(map(id2fname, v3-tags["simple_background"]))

In [ ]:
Exception